In [ ]:
from breeze_connect import BreezeConnect
import pandas as pd
import time
import os
from datetime import datetime, date, timedelta
import functions as f
import ast
from selenium import webdriver
from pyotp import TOTP
import urllib
import json

In [ ]:
line = ast.literal_eval(f.get_line('login_credential.txt', 8).strip())
date = line['date']

today = datetime.now().date()
today_date = f'{today.day}-{today.month}-{today.year}'

if today_date != date:

    def autologin():

        api_key = str(ast.literal_eval(f.get_line('login_credential.txt', 1).strip())['api_key'])
        api_secret = str(ast.literal_eval(f.get_line('login_credential.txt', 2).strip())['api_secret'])
        password = str(ast.literal_eval(f.get_line('login_credential.txt', 4).strip())['password'])
        userID = str(ast.literal_eval(f.get_line('login_credential.txt', 5).strip())['userID'])
        totp = str(ast.literal_eval(f.get_line('login_credential.txt', 6).strip())['totp'])

        # ----------------- Selenium -----------------

        browser = webdriver.Chrome()
        browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
        browser.implicitly_wait(5)
        breeze = BreezeConnect(api_key=api_key)
        username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
        password_field = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input')

        username.send_keys(userID)
        password_field.send_keys(password)

        # Checkbox
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
        # Click Login Button
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
        time.sleep(2)
        pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')

        totp = TOTP(totp)
        token = totp.now()
        pin.send_keys(token)
        browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
        time.sleep(1)

        # Check if 'apisession=' exists in the URL
        if 'apisession=' in browser.current_url:
            temp_token = browser.current_url.split('apisession=')[1][:8]
            # Save in Database or text File
            print('temp_token', temp_token)
            f.update_line('login_credential.txt', 7, {'session_key': temp_token})

            breeze.generate_session(api_secret=api_secret, session_token=temp_token)
            print(breeze.get_funds())
        else:
            print("Error: 'apisession=' not found in the URL.")

        browser.quit()

    autologin()

    f.update_line('login_credential.txt', 8, {'date': today_date})


In [ ]:
api_key = str(ast.literal_eval(f.get_line('login_credential.txt', 1).strip())['api_key'])
api_secret = str(ast.literal_eval(f.get_line('login_credential.txt', 2).strip())['api_secret'])
session_key = str(ast.literal_eval(f.get_line('login_credential.txt', 7).strip())['session_key'])

breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret, session_token=session_key)

# print(breeze.get_funds())


In [ ]:
# ----------------------------------- Save Masterlist file ----------------------------------- #

# import requests
# import zipfile
# import os

# def download_security_master():
#     url = "https://directlink.icicidirect.com/NewSecurityMaster/SecurityMaster.zip"
#     response = requests.get(url)

#     if response.status_code == 200:
#         if not os.path.exists('master_list_files'):
#             os.makedirs('master_list_files')

#         zip_path = os.path.join('master_list_files', 'SecurityMaster.zip')
#         with open(zip_path, 'wb') as f:
#             f.write(response.content)

#         with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#             zip_ref.extractall('master_list_files')

#         txt_files = [f for f in os.listdir('master_list_files') if f.endswith('.txt')]

#         for txt_file in txt_files:
#             txt_path = os.path.join('master_list_files', txt_file)
#             csv_path = os.path.join('master_list_files', txt_file.replace('.txt', '.csv'))

#             os.rename(txt_path, csv_path)

#         print("Security master file downloaded and extracted successfully")

# # Execute the download
# download_security_master()

In [ ]:
# # ----------------------------------- Find expiry dates ----------------------------------- #

# df_monday = f.get_monday_timestamps()
# df_range = pd.read_csv('nifty50_day_range.csv')

# expiry_dates_list_code = ['2020-01-02T07:00:00.000Z']
# row_not_found = []

# for i in range(-len(df_monday), 0):
#     print(i, df_monday['start_time'].iloc[i], df_monday['end_time'].iloc[i+1])

#     test_date = df_monday['start_time'].iloc[i]
#     next_dates = f.get_next_five_dates(test_date)

#     date_only = datetime.strptime(test_date, '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d')

#     index, row = f.get_row_by_date(df_range, date_only)

#     if index != None:
#         range_is = f.generate_range_for_expiry_date(f.get_nearest_midpoint(row))
#     else:
#         row_not_found.append(test_date)

#     for n in range(-len(next_dates), 0):
#         print(n, next_dates[n])

#         for z in range(0, len(range_is)):

#             response = breeze.get_historical_data_v2(interval="1second",
#                         from_date= df_monday['start_time'].iloc[i],
#                         to_date= df_monday['end_time'].iloc[i+1],
#                         stock_code="NIFTY",
#                         exchange_code="NFO",
#                         product_type="options",
#                         expiry_date=next_dates[n],
#                         right="call",
#                         strike_price=str(range_is[z]))

#             if response['Success'] != []:
#                 expiry_dates_list_code.append(next_dates[n])
#                 print(f"Expiry date : {next_dates[n]}\n")
#                 break

#             time.sleep(0.05)

#         break

# print("\nSome dates not found in the range csv file : ", row_not_found)

# with open('expiry_dates_list_code.json', 'w') as file:
#     json.dump(expiry_dates_list_code, file, indent=2)

In [ ]:
def download_strike_data_with_internet_connection(intervals, expiry_date, right, strike_price):

    internet_status = True

    while internet_status:
        try:

            status_list = []
            strike_data_status = 'Success'

            df = pd.DataFrame()

            for i in range(0, len(intervals)-1):
                # print(intervals[i], intervals[i+1])

                response = breeze.get_historical_data_v2(interval="1second",
                            from_date= intervals[i],
                            to_date= intervals[i+1],
                            stock_code="NIFTY",
                            exchange_code="NFO",
                            product_type="options",
                            expiry_date=expiry_date,
                            right=right,
                            strike_price=strike_price)

                if response['Success'] == []:
                    status_list.append(f"Failure {intervals[i]}, {intervals[i+1]}, {expiry_date}, {right}, {strike_price}")     # diagnose
                    strike_data_status = 'Failure'                                                                              # diagnose

                tik_df = pd.DataFrame(response['Success'])
                df = pd.concat([df, tik_df], ignore_index=True)

            return df, status_list, strike_data_status

        except Exception as e:
            # Check if the error is due to the missing attribute in BreezeConnect
            if "'BreezeConnect' object has no attribute 'error_exception'" in str(e):
                print("internet connection is lost")
                time.sleep(2)
                internet_status = f.is_internet_available()
            else:
                print(f"An unexpected error occurred: {e}")

In [ ]:
def check_strike_with_internet_connection(intervals, expiry_date, right, strike_price):

    internet_status = True

    while internet_status:
        try:

            response = breeze.get_historical_data_v2(
                interval="1second",
                from_date=intervals[0],
                to_date=intervals[1],
                stock_code="NIFTY",
                exchange_code="NFO",
                product_type="options",
                expiry_date=expiry_date,
                right=right,
                strike_price=strike_price)

            return response

        except Exception as e:
            # Check if the error is due to the missing attribute in BreezeConnect
            if "'BreezeConnect' object has no attribute 'error_exception'" in str(e):
                print("internet connection is lost")
                time.sleep(2)
                internet_status = f.is_internet_available()
            else:
                print(f"An unexpected error occurred: {e}")


In [ ]:
def get_day_data_to_files():

    yesterday = today - timedelta(days=1)
    yesterday_date = yesterday.strftime('%Y-%m-%d')

    df_range = pd.read_csv('nifty50_day_range.csv')
    last_date = df_range['date'].iloc[-1]

    if last_date < yesterday_date:

        def generate_dates_except_today(last_date):
            start_date = datetime.strptime(last_date, '%Y-%m-%d')
            today = datetime.now()

            date_list = []
            current_date = start_date

            while current_date.date() < today.date():
                date_list.append(current_date.strftime('%Y-%m-%d'))
                current_date += timedelta(days=1)

            return date_list

        remaining_date_list = generate_dates_except_today(last_date)

        response = breeze.get_historical_data_v2(
            interval="1day",
            from_date=remaining_date_list[0]+'T07:00:00.000Z',
            to_date=remaining_date_list[len(remaining_date_list)-1]+'T07:00:00.000Z',
            stock_code="NIFTY",
            exchange_code="nse",
            product_type="options",
            expiry_date=0,
            right=0,
            strike_price=0)

        if response['Success'] != []:

            df = pd.DataFrame()

            tik_df = pd.DataFrame(response['Success'])
            df = pd.concat([df, tik_df], ignore_index=True)
            df = df.rename(columns={'stock_code': 'symbol'})
            df = df[['symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume']]

            df['symbol'] = df['symbol'].replace('NIFTY', 'NIFTY50')
            df['datetime'] = pd.to_datetime(df['datetime']) + pd.Timedelta(hours=5, minutes=30)

            existing_df = pd.read_csv('NIFTY50.csv')
            existing_df['datetime'] = pd.to_datetime(existing_df['datetime'])

            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df = combined_df.drop_duplicates(subset='datetime', keep='last')

            combined_df = combined_df.sort_values('datetime')

            combined_df.to_csv('NIFTY50.csv', index=False)

            df = pd.read_csv('NIFTY50.csv')

            df['datetime'] = pd.to_datetime(df['datetime'])
            result_df = pd.DataFrame({
                'date': df['datetime'].dt.strftime('%Y-%m-%d'),
                'high_plus_1000': df['high'] + 1000,
                'low_minus_1000': df['low'] - 1000
            })
            result_df.to_csv('nifty50_day_range.csv', index=False)

get_day_data_to_files()

In [ ]:
# --------------------------- Download second by second data in day file --------------------------- #

right = "call"
dates = f.get_all_weekdays(datetime(2025, 1, 1))
df_range = pd.read_csv('nifty50_day_range.csv')

status_list_outside_main = []
with open('status_list_outside_main.json', 'r') as file:
    status_list_outside_main = json.load(file)

for i in range(-len(dates), 0):
    print(dates[i])

    status_list_inside_main = []
    strike_data_status_inside_main = []

    main_df = pd.DataFrame()

    expiry_date = f.find_next_expiry_date(dates[i])

    intervals = f.generate_15_min_intervals(dates[i])

    date_only = datetime.strptime(dates[i], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d')
    index, row = f.get_row_by_date(df_range, date_only)

    range_is = f.generate_symmetric_range(f.get_nearest_midpoint(row), 50, 10)

    greater_side = False
    lesser_side = False
    greater_n = 0
    lesser_n = 0

    current_date = datetime.strptime(dates[i], '%Y-%m-%dT%H:%M:%S.%fZ').date()
    file_name = f'D:/Offline Data/Original Data/Nifty_1_Second_ICICI/{current_date.year}_{current_date.month}_{current_date.day}_nifty_option_data_1second.csv'

    is_date_found = f.check_date_in_records(date_only)
    if is_date_found == "date_not_found":

        strike_data_status_inside_main.append(date_only)

        for z in range(0, len(range_is)):
            # print(range_is[z])

            if z == 0:
                # print("Center value", range_is[z])

                response = check_strike_with_internet_connection(intervals=intervals,
                                                                expiry_date=expiry_date,
                                                                right="call",
                                                                strike_price=str(range_is[z]))

                if response['Success'] != []:
                    df, status_list, strike_data_status = download_strike_data_with_internet_connection(intervals, expiry_date, right, strike_price = str(range_is[z]))
                    main_df = pd.concat([main_df, df], ignore_index=True)

                    status_list_inside_main = status_list_inside_main + status_list
                    strike_data_status_inside_main.append(strike_data_status)

                # print(response)

            # ---------------------- Greater side ---------------------- #

            if range_is[z] > range_is[0] and not greater_side:
                # print("Greater side", range_is[z])

                response = check_strike_with_internet_connection(intervals=intervals,
                                                                expiry_date=expiry_date,
                                                                right="call",
                                                                strike_price=str(range_is[z]))

                if response['Success'] != []:
                    df, status_list, strike_data_status = download_strike_data_with_internet_connection(intervals, expiry_date, right, strike_price = str(range_is[z]))
                    main_df = pd.concat([main_df, df], ignore_index=True)

                    status_list_inside_main = status_list_inside_main + status_list
                    strike_data_status_inside_main.append(strike_data_status)

                else:
                    greater_n += 1
                    if greater_n == 4:
                        greater_side = True

                if greater_n > 0 and response['Success'] != []:
                    greater_n -= 1

                # print(response)

            # ---------------------- Lesser side ---------------------- #

            if range_is[z] < range_is[0] and not lesser_side:
                # print("Lesser side", range_is[z])

                response = check_strike_with_internet_connection(intervals=intervals,
                                                                expiry_date=expiry_date,
                                                                right="call",
                                                                strike_price=str(range_is[z]))

                if response['Success'] != []:
                    df, status_list, strike_data_status = download_strike_data_with_internet_connection(intervals, expiry_date, right, strike_price = str(range_is[z]))
                    main_df = pd.concat([main_df, df], ignore_index=True)

                    status_list_inside_main = status_list_inside_main + status_list
                    strike_data_status_inside_main.append(strike_data_status)

                else:
                    lesser_n += 1
                    if lesser_n == 4:
                        lesser_side = True

                if lesser_n > 0 and response['Success'] != []:
                    lesser_n -= 1

                # print(response)

        main_df.to_csv(file_name, index=False)
        f.update_date_records(date_only, str(datetime.now().date()))

        if "Failure" in strike_data_status_inside_main:
            failure_indices = [i for i, status in enumerate(strike_data_status_inside_main) if status == "Failure"]
            print(f"Found Failure at indices: {failure_indices}")
        else:
            print("Successful data download for all strikes :", date_only)

        status_list_outside_main = status_list_outside_main + status_list_inside_main

        with open('status_list_outside_main.json', 'w') as file:
            json.dump(status_list_outside_main, file, indent=2)

    # if -len(dates) + 3 == i:
    #     break


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
right = "call"
dates = f.get_all_weekdays(datetime(2025, 1, 1))
date = dates[26]
intervals = f.generate_15_min_intervals(date)
expiry_date = '2025-02-06T07:00:00.000Z'
strike_price = "21850"
n = 10
right, date

In [ ]:
response = breeze.get_historical_data_v2(
    interval="1second",
    from_date=intervals[n],
    to_date=intervals[n+1],
    stock_code="NIFTY",
    exchange_code="NFO",
    product_type="options",
    expiry_date=expiry_date,
    right=right,
    strike_price=strike_price)

n += 1
response, intervals[n], intervals[n+1], expiry_date, right, strike_price